In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb

keras = tf.keras

print("Tensorflow Version: " + tf.VERSION)

Tensorflow Version: 1.13.1


# IMDB Dataset 
https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

In [2]:
# imdb data
INDEX_FROM = 0

(x_train, y_train), (x_test, y_test) = imdb.load_data(index_from=INDEX_FROM)

word_index = imdb.get_word_index()
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2

word_index_inv = {v: k for k, v in word_index.items()}

def encoded_vector2sentence(vec, word_index_inv = word_index_inv):
    sentence = map(lambda i: word_index_inv[i], vec)
    return " ".join(sentence)

def pad_input(vec, maxlen):
    return keras.preprocessing.sequence.pad_sequences(vec, maxlen=maxlen, padding='post')

def sentence2encoded_vec(sentence):
    sentence_arr = sentence.split(" ")
    return list(map(lambda word: word_index[word.lower()], sentence_arr))

In [3]:
import os

# Pretrained Word Embedding
GLOVE_DIR = "D:/google drive/haw/master/mastertheisis/hauptprojekt"
EMBEDDING_DIM = 50

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.50d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [4]:
# match the word index with the word_embedding index
import re

regex = re.compile('[^a-zA-Z0-9]')
def normalize_word(word):
    #return regex.sub('', word)
    return word.replace("'d", "").replace("'s", "").replace("'", "")

n_unknown_words = 0
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(normalize_word(word))
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        n_unknown_words += 1

print("{}/{} unknown words".format(n_unknown_words, len(word_index)))

16278/88587 unknown words


In [55]:
# raw data representation
print(x_train[0], y_train[0])

[1, 11, 19, 13, 40, 527, 970, 1619, 1382, 62, 455, 4465, 63, 3938, 1, 170, 33, 253, 2, 22, 97, 40, 835, 109, 47, 667, 22662, 6, 32, 477, 281, 2, 147, 1, 169, 109, 164, 21628, 333, 382, 36, 1, 169, 4533, 1108, 14, 543, 35, 10, 444, 1, 189, 47, 13, 3, 144, 2022, 16, 11, 19, 1, 1917, 4610, 466, 1, 19, 68, 84, 9, 13, 40, 527, 35, 73, 12, 10, 1244, 1, 19, 14, 512, 14, 9, 13, 623, 15, 19190, 2, 59, 383, 9, 5, 313, 5, 103, 2, 1, 2220, 5241, 13, 477, 63, 3782, 30, 1, 127, 9, 13, 35, 616, 2, 22, 121, 48, 33, 132, 45, 22, 1412, 30, 3, 19, 9, 212, 25, 74, 49, 2, 11, 404, 13, 79, 10308, 5, 1, 104, 114, 5949, 12, 253, 1, 31047, 4, 3763, 2, 720, 33, 68, 40, 527, 473, 23, 397, 314, 43, 4, 1, 12115, 1026, 10, 101, 85, 1, 378, 12, 294, 95, 29, 2068, 53, 23, 138, 3, 191, 7483, 15, 1, 223, 19, 18, 131, 473, 23, 477, 2, 141, 27, 5532, 15, 48, 33, 25, 221, 89, 22, 101, 1, 223, 62, 13, 35, 1331, 85, 9, 13, 280, 2, 13, 4469, 110, 100, 29, 12, 13, 5342, 16, 175, 29] 1


In [56]:
# parsed sentence
print(encoded_vector2sentence(x_train[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited <START> part they played <UNK> you could just imagine being there robert redford's is an amazing actor <UNK> now <START> same being director norman's father came from <START> same scottish island as myself so i loved <START> fact there was a real connection with this film <START> witty remarks throughout <START> film were great it was just brilliant so much that i bought <START> film as soon as it was released for retail <UNK> would recommend it to everyone to watch <UNK> <START> fly fishing was amazing really cried at <START> end it was so sad <UNK> you know what they say if you cry at a film it must have been good <UNK> this definitely was also congratulations to <START> two little boy's that played <START> part's of norman <UNK> paul they were just brilliant children are often left out of <START> praising list i think because <START> stars that play them all grown up are such a big

In [5]:
# Max Input Length
max_length = max(map(lambda vec: len(vec), x_train + x_test))

In [6]:
# pad input vectors
x_train_padded = pad_input(x_train, max_length)
x_test_padded = pad_input(x_test, max_length)
x_train_padded[0]

array([ 1, 11, 19, ...,  0,  0,  0])

In [27]:
# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

embedding_layer = keras.layers.Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

model = keras.Sequential([
    embedding_layer,
    keras.layers.Dropout(0.25),
    keras.layers.Conv1D(64, 15, activation="relu"),
    keras.layers.Dropout(0.25),
    keras.layers.Conv1D(64, 5, activation="relu"),
    keras.layers.MaxPooling1D(35),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 2697, 50)          4429400   
_________________________________________________________________
dropout_17 (Dropout)         (None, 2697, 50)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 2683, 64)          48064     
_________________________________________________________________
dropout_18 (Dropout)         (None, 2683, 64)          0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 2679, 64)          20544     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 76, 64)            0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 76, 64)            0         
__________

In [28]:
tb_callback = keras.callbacks.TensorBoard(
    log_dir='./logs', 
    histogram_freq=0,
    write_graph=True, 
    write_images=True) 

model.fit(x_train_padded, y_train, validation_split=0.05, epochs=8, callbacks=[tb_callback])
loss, accuracy = model.evaluate(x_train_padded, y_train)

Train on 23750 samples, validate on 1250 samples
Epoch 1/8
23750/23750 [==============================] - 22s 910us/sample - loss: 0.6118 - acc: 0.6446 - val_loss: 0.5605 - val_acc: 0.7368
Epoch 2/8
23750/23750 [==============================] - 21s 886us/sample - loss: 0.5058 - acc: 0.7567 - val_loss: 0.4876 - val_acc: 0.7664
Epoch 3/8
23750/23750 [==============================] - 21s 884us/sample - loss: 0.4722 - acc: 0.7777 - val_loss: 0.4653 - val_acc: 0.7792
Epoch 4/8
23750/23750 [==============================] - 21s 887us/sample - loss: 0.4479 - acc: 0.7950 - val_loss: 0.4315 - val_acc: 0.8040
Epoch 5/8
23750/23750 [==============================] - 21s 887us/sample - loss: 0.4280 - acc: 0.8048 - val_loss: 0.3965 - val_acc: 0.8480
Epoch 6/8
23750/23750 [==============================] - 21s 892us/sample - loss: 0.4116 - acc: 0.8128 - val_loss: 0.4121 - val_acc: 0.8224
Epoch 7/8
23750/23750 [==============================] - 21s 873us/sample - loss: 0.3975 - acc: 0.8211 - val_lo

In [29]:
def test_model(x, y):
    test_result = np.round(model.predict(x))
    test_errors = np.squeeze(test_result) != y
    correct_percentage = np.sum(test_errors) / len(y)
    print("%i / %i (%.2f%%) are correct" % (len(y) - np.sum(test_errors), len(y), 100 * (1 - correct_percentage)))

In [30]:
test_model(x_test_padded, y_test)
test_model(x_train_padded, y_train)

20551 / 25000 (82.20%) are correct
21360 / 25000 (85.44%) are correct


In [34]:
custom_sentences = [
    "this was a very bad movie",
    "this was a very good movie",
    "I did not like this movie at all",
    "I hope there will be a sequal",
    "not bad",
    "bad",
    "not good",
    "one of the best movies of the year",
    "the first part was bad but the second part got better",
    "the first part was not bad but after that it just got worse",
    "this film was just brilliant casting location scenery story direction everyone's really suited part they played you could just imagine being there robert redford's is an amazing actor now same being director norman's father came from same scottish island as myself so i loved fact there was a real connection with this film witty remarks throughout film were great it was just brilliant so much that i bought"
]


def create_rating(sentences):
    encoded_custom_sentences = pad_input(list(map(sentence2encoded_vec, custom_sentences)), max_length)
    custom_result = model.predict(encoded_custom_sentences)

    result = []
    for i in range(len(custom_sentences)):
        sentence = custom_sentences[i]
        prediction = custom_result[i][0]
        rating = int(round(prediction * 10))
        result += [(sentence, prediction, rating)]
    return result
                  

for (sentence, prediction, rating) in create_rating(custom_sentences):
    print("%s (%.2f%%)\n%s\n" % (rating * "⭐", prediction * 100, sentence)) 

⭐⭐ (17.76%)
this was a very bad movie

⭐⭐⭐⭐⭐⭐⭐⭐⭐ (92.86%)
this was a very good movie

⭐⭐⭐⭐⭐⭐⭐ (71.37%)
I did not like this movie at all

⭐⭐⭐⭐⭐⭐⭐ (65.06%)
I hope there will be a sequal

⭐⭐⭐⭐⭐ (52.46%)
not bad

⭐⭐⭐⭐⭐⭐ (62.59%)
bad

⭐⭐⭐⭐⭐⭐⭐ (67.94%)
not good

⭐⭐⭐⭐⭐⭐⭐⭐⭐ (89.08%)
one of the best movies of the year

⭐⭐⭐⭐ (41.84%)
the first part was bad but the second part got better

⭐ (14.06%)
the first part was not bad but after that it just got worse

⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐ (96.61%)
this film was just brilliant casting location scenery story direction everyone's really suited part they played you could just imagine being there robert redford's is an amazing actor now same being director norman's father came from same scottish island as myself so i loved fact there was a real connection with this film witty remarks throughout film were great it was just brilliant so much that i bought

